# Open Demand

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## Import Database

In [2]:
#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## Define clean_up

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## Import Tables

In [4]:
# Inventory
inventory_sql = """
SELECT 
    ItemCode AS isbn
    , WhsCode AS po_whse
    , AVG(OnHand) AS Onhand
FROM 
	SAP.InventoryReportView
WHERE
	WhsCode IN ('WW', 'AW')
	AND OnHand > 0
GROUP BY
	ItemCode
	, WhsCode
"""

# Line Purchase Order
PO_sql = """
WITH PO AS
(SELECT 
	CASE
		WHEN Rate = 0 THEN 1
		ELSE Rate
	END AS EXCHANGE
	, ISBN
	, OpenQuantity
	, GrossPrice
	, [Discount]
	, ShippingCostUSD
	, Whse
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [DocumentStatus]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
FROM [SAP].[LinePurchaseOrderView]
)

SELECT
	[ISBN]
	, OpenQuantity AS po_open_qty
	, (GrossPrice*EXCHANGE*(1-[Discount]/100) +ShippingCostUSD) AS LandedCost
	, [Whse] AS po_whse
    , [RowDeliveryDate]
    , [DocumentNumber] AS po_number
    , [DocumentName] AS po_name
    , [BPCode] AS supplier_code
    , [BPName] AS supplier
    , [OrderStatus]
	, (CASE
		WHEN [RowDeliveryDate] > GETDATE() THEN 'OnTime'
		ELSE 'Late'
	END) AS po_date_status
FROM 
	PO
WHERE 
	Whse in ('WW', 'TB') 
	AND OpenQuantity > 0
"""


# Line Sales Order


SO_sql = """
SELECT 
	Isbn
	, OpenQuantity AS so_open_qty
	, UnitPrice AS SalePrice
	, WarehouseCode AS so_whse
	, DeliveryDate AS customer_drop_date
	, DocumentNumber AS so_number
	, DocumentName AS so_name
	, CustomerOrVendorCode AS customer_code
	, CustomerOrVendorName AS customer
	, CustomerRefNo
	, CustomerId
	, Condition
	, (CASE
			WHEN DeliveryDate > GETDATE() THEN 'OnTime'
			ELSE 'Late'
		END) AS so_date_status
FROM
	[SAP].[SaleOrderReportView]
WHERE OpenQuantity > 0 AND WarehouseCode IN ('WW')
"""


receiving_sql = """
WITH RECEIVED AS
(SELECT
	CASE
		WHEN x_Rate = 0 THEN 1
		ELSE x_Rate
	END AS EXCHANGE
	, [ISBN]
	, [Price]
	, [Discount]
	, [ShippingCost]
	, [WarehouseCode]
	, [PostingDate]
    , [DocName]
	, [DocNum]
	, [BPCode]
	, [BPName]
FROM [SAP].[GoodsReceiptReportView]
)

SELECT
	[ISBN]
	, ([Price]*[EXCHANGE]*(1-[Discount]/100) + [ShippingCost]) AS landed_cost
	, [WarehouseCode] AS po_whse
	, [PostingDate] AS received_date
    , [DocNum] AS po_number 
	, [DocName] AS po_name
	, [BPCode] AS supplier_code
	, [BPName] AS supplier
  FROM RECEIVED
  WHERE WarehouseCode = 'WW'
  ORDER BY PostingDate desc
  """

In [5]:
inventory = pd.read_sql(inventory_sql, prod_db)
inventory.columns = map(str.lower, inventory.columns)
for col in ['isbn', 'po_whse']:
    inventory[col] = inventory.apply(lambda x: clean_up(x[col]), axis =1)
print(inventory.isna().sum())
inventory.head()

isbn       0
po_whse    0
onhand     0
dtype: int64


,isbn,po_whse,onhand
0,9780071637916,AW,5
1,9780071717595,AW,20
2,9780071740289,AW,55
3,9780071744416,AW,30
4,9780071754422,AW,10


In [6]:
receiving = pd.read_sql(receiving_sql, prod_db)
receiving.dropna(inplace=True)
receiving.columns = map(str.lower, receiving.columns)
receiving['isbn'] = receiving['isbn'].astype(str)
for col in ['isbn', 'po_whse', 'po_name', 'supplier_code', 'supplier']:
    receiving[col] = receiving.apply(lambda x: clean_up(x[col]), axis =1)
print(receiving.isna().sum())
print(receiving.dtypes)
receiving.head()

isbn             0
landed_cost      0
po_whse          0
received_date    0
po_number        0
po_name          0
supplier_code    0
supplier         0
dtype: int64
isbn                     object
landed_cost             float64
po_whse                  object
received_date    datetime64[ns]
po_number                 int64
po_name                  object
supplier_code            object
supplier                 object
dtype: object


,isbn,landed_cost,po_whse,received_date,po_number,po_name,supplier_code,supplier
0,9780357038314,71.632634,WW,2020-06-15,17570,2020.02.03.COINFO.HEATB297,V1412,BOOKROCK PRT LTD
1,9781305956780,39.698020,WW,2020-06-15,17575,2020.04.28.HEAT.SHIPMENT.20THAPRIL.OVERAGES,V1412,BOOKROCK PRT LTD
2,9780137043293,25.072427,WW,2020-06-15,17578,2020.02.24.COINFO.HEATB301,V1412,BOOKROCK PRT LTD
3,9781337558495,54.978325,WW,2020-06-15,17579,2020.02.20.COINFO.HEATB300,V1412,BOOKROCK PRT LTD
4,9780357038314,71.632634,WW,2020-06-15,17580,2020.02.03.COINFO.HEATB297,V1412,BOOKROCK PRT LTD


In [7]:
inventory_ww = inventory[inventory['po_whse'] == 'WW']
inventory_ww = inventory_ww.merge(receiving, how='left', on=['isbn', 'po_whse'] )
inventory_ww.sort_values('received_date', ascending=False, inplace=True, ignore_index=True)
inventory_ww = inventory_ww.groupby(['isbn'])['onhand', 'landed_cost', 'po_whse','received_date', 'po_number', 'po_name', 'supplier_code', 'supplier'].first().reset_index()
inventory_ww.head()

,isbn,onhand,landed_cost,po_whse,received_date,po_number,po_name,supplier_code,supplier
0,9780132155038,1,62.906548,WW,2020-06-15,17560,2020.01.27.COINFO.HEATB296.SEA,V1412,BOOKROCK PRT LTD
1,9780132576246,2,81.904200,WW,2020-06-15,17582,2019.12.11.COINFO.HEAT316.SEA,V1412,BOOKROCK PRT LTD
2,9780132978729,1,85.965620,WW,2020-06-15,17559,2019.12.11.COINFO.HEAT316.SEA,V1412,BOOKROCK PRT LTD
3,9780133803839,2,74.725611,WW,2020-06-15,17592,2019.12.19.COINFO.HEAT317.SEA,V1412,BOOKROCK PRT LTD
4,9780134054285,5,65.228992,WW,2020-06-15,17581,2019.12.19.COINFO.HEAT317.SEA,V1412,BOOKROCK PRT LTD


In [8]:
po = pd.read_sql(PO_sql, prod_db)
po.columns = map(str.lower, po.columns)
for col in ['isbn', 'po_whse', 'po_name', 'supplier_code', 'supplier', 'orderstatus', 'po_date_status']:
    po[col] = po.apply(lambda x: clean_up(x[col]), axis =1)
print(po.isna().sum())
po.sort_values(['rowdeliverydate', 'po_whse', 'landedcost'], ascending=[True,False,True], inplace=True, ignore_index=True)
po.head()

isbn               0
po_open_qty        0
landedcost         0
po_whse            0
rowdeliverydate    0
po_number          0
po_name            0
supplier_code      0
supplier           0
orderstatus        0
po_date_status     0
dtype: int64


,isbn,po_open_qty,landedcost,po_whse,rowdeliverydate,po_number,po_name,supplier_code,supplier,orderstatus,po_date_status
0,9781473924994,1.0,19.219698,WW,2020-04-29,1694,200217 ALEK SAGE (GBP) PO,V1104,ALEK,PROCESSED,LATE
1,9781473912991,1.0,21.740658,WW,2020-04-29,1694,200217 ALEK SAGE (GBP) PO,V1104,ALEK,PROCESSED,LATE
2,9781446254790,1.0,21.740658,WW,2020-04-29,1694,200217 ALEK SAGE (GBP) PO,V1104,ALEK,PROCESSED,LATE
3,9781473902190,1.0,23.001138,WW,2020-04-29,1694,200217 ALEK SAGE (GBP) PO,V1104,ALEK,PROCESSED,LATE
4,9781446267097,1.0,23.001138,WW,2020-04-29,1694,200217 ALEK SAGE (GBP) PO,V1104,ALEK,PROCESSED,LATE


In [9]:
so = pd.read_sql(SO_sql, prod_db)
so.columns = map(str.lower, so.columns)
for col in ['isbn', 'so_whse', 'so_name', 'customer', 'customer_code', 'condition', 'so_date_status']:
    so[col] = so.apply(lambda x: clean_up(x[col]), axis =1)
print(so.isna().sum())
so.sort_values(['saleprice', 'customer_drop_date'], ascending=[False,True], inplace=True, ignore_index=True)
so.head()

isbn                    0
so_open_qty             0
saleprice               0
so_whse                 0
customer_drop_date      0
so_number               0
so_name                 0
customer_code           0
customer                0
customerrefno           2
customerid            707
condition               0
so_date_status          0
dtype: int64


,isbn,so_open_qty,saleprice,so_whse,customer_drop_date,so_number,so_name,customer_code,customer,customerrefno,customerid,condition,so_date_status
0,9781305575080,3,223.17,WW,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME
1,9780840068330,2,195.27,WW,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME
2,9780134641690,2,189.71,WW,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME
3,9780133953145,8,184.77,WW,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME
4,9780134475585,5,182.27,WW,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME


In [10]:
so.columns

Index(['isbn', 'so_open_qty', 'saleprice', 'so_whse', 'customer_drop_date',
       'so_number', 'so_name', 'customer_code', 'customer', 'customerrefno',
       'customerid', 'condition', 'so_date_status'],
      dtype='object')

In [11]:
#ALLOCATE RECEIVED inventory_ww INTO THE 
inv_so = pd.DataFrame()
inventory_ww.reset_index(inplace = True, drop = True)
for so_id in list(so[so['isbn'].isin(list(inventory_ww['isbn']))].index.values):
    for inv_id in list(inventory_ww[inventory_ww['isbn'].isin(list(so['isbn']))].index.values):
        if (inventory_ww.iloc[inv_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (inventory_ww.iloc[inv_id, :]['onhand'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0):
            row_inv_so = pd.merge(inventory_ww.iloc[[inv_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_inv_so['allocate_qty'] = row_inv_so[['onhand','so_open_qty']].min( axis = 1, skipna = True)
            inventory_ww.loc[inv_id,'onhand'] = inventory_ww.loc[inv_id,'onhand'] - row_inv_so.iloc[0,:]['allocate_qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_inv_so.iloc[0,:]['allocate_qty']
            row_inv_so['orderstatus'] = 'READY TO SHIP'
            row_inv_so['inventory_ww_date_status'] = so.iloc[so_id, :]['so_date_status']
            inv_so = inv_so.append(row_inv_so)


In [12]:
inv_so.head()

,isbn,onhand,landed_cost,po_whse,received_date,po_number,po_name,supplier_code,supplier,so_open_qty,...,so_name,customer_code,customer,customerrefno,customerid,condition,so_date_status,allocate_qty,orderstatus,inventory_ww_date_status
0,9780134871400,12,92.492790,WW,2020-06-15,17596,2020.01.17.COINFO.HEATB292.SEA,V1412,BOOKROCK PRT LTD,2,...,2020.01.02.MBS.PEARSON,C1032,MBS TEXTBOOKS,BX20200103-P (re),520847,NEW,LATE,2,READY TO SHIP,LATE
0,9780134871400,10,92.492790,WW,2020-06-15,17596,2020.01.17.COINFO.HEATB292.SEA,V1412,BOOKROCK PRT LTD,7,...,2019.12.23.MBS.GUIDE,C1032,MBS TEXTBOOKS,BX20191223 (re),505430,NEW,LATE,7,READY TO SHIP,LATE
0,9780134871400,3,92.492790,WW,2020-06-15,17596,2020.01.17.COINFO.HEATB292.SEA,V1412,BOOKROCK PRT LTD,45,...,2019.12.05.MBS.PEARSON,C1032,MBS TEXTBOOKS,BX20191211 (re).,540943,NEW,LATE,3,READY TO SHIP,LATE
0,9780132978729,1,85.965620,WW,2020-06-15,17559,2019.12.11.COINFO.HEAT316.SEA,V1412,BOOKROCK PRT LTD,5,...,2019.12.05.MBS.PEARSON,C1032,MBS TEXTBOOKS,BX20191211 (re).,518042,NEW,LATE,1,READY TO SHIP,LATE
0,9781337401043,2,92.443475,WW,2020-06-15,17589,2020.03.02.COINFO.HEATB304,V1412,BOOKROCK PRT LTD,2,...,2020.02.20.MBS.GUIDE,C1032,MBS TEXTBOOKS,BX20200220,596888,NEW,LATE,2,READY TO SHIP,LATE


In [13]:
# po_so = pd.DataFrame()
# for so_id in list(so[so['isbn'].isin(list(po['isbn'])) & (so['so_open_qty'] > 0)].index.values):
#     for po_id in list(po[po['isbn'].isin(list(so['isbn'])) & (po['po_open_qty'] > 0)].index.values):
#         if (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) >= 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH POSITIVE COMMISSION
#             row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
#             po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
#             # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
#             # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # else:
#             #     row_po_so_good['action'] = 'PROFIT'
#         elif (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) < 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH NEGATIVE COMMISSION
#             row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
#             po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
#             # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
#             # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # else:
#             #     row_po_so_good['action'] = 'NO PROFIT'
#         po_so = po_so_good.append(row_po_so_good)

In [14]:
po_so = pd.DataFrame()
for so_id in list(so[so['isbn'].isin(list(po['isbn'])) & (so['so_open_qty'] > 0)].index.values):
    for po_id in list(po[po['isbn'].isin(list(so['isbn'])) & (po['po_open_qty'] > 0)].index.values):
        if (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and ((so.iloc[so_id, :]['so_open_qty'] > 0) and (so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) > -5):
            row_po_so = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_po_so['qty'] = row_po_so[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
            po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so.iloc[0,:]['qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so.iloc[0,:]['qty']
            po_so = po_so.append(row_po_so)

In [15]:
po_so.head()

,isbn,po_open_qty,landedcost,po_whse,rowdeliverydate,po_number,po_name,supplier_code,supplier,orderstatus,...,customer_drop_date,so_number,so_name,customer_code,customer,customerrefno,customerid,condition,so_date_status,qty
0,9781305575080,3.0,51.867356,WW,2020-07-28,2106,2020.06.01.ALEK.CENGAGE.GBP,V1104,ALEK,PROCESSED,...,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME,3.0
0,9780840068330,2.0,66.865832,WW,2020-07-28,2106,2020.06.01.ALEK.CENGAGE.GBP,V1104,ALEK,PROCESSED,...,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME,2.0
0,9781305859975,3.0,49.140360,WW,2020-07-28,2106,2020.06.01.ALEK.CENGAGE.GBP,V1104,ALEK,PROCESSED,...,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME,3.0
0,9780134292816,10.0,127.600640,WW,2020-07-20,2123,2020.06.01.WESTERNCAMPUS.ORDER24,V4069,WESTERNCAMPUS,PROCESSED,...,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME,10.0
0,9780134414232,3.0,127.600640,WW,2020-07-20,2123,2020.06.01.WESTERNCAMPUS.ORDER24,V4069,WESTERNCAMPUS,PROCESSED,...,2020-07-31,19651,2020.05.08.AKADEMOS.GUIDE,C1052,AKADEMOS,bxchg050820d,None,NEW,ONTIME,3.0


In [16]:
"""
RETAIL AW WAREHOUSE"

"""

# inv_aw = inventory[inventory['po_whse'] == 'AW']
# inv_aw.reset_index(inplace = True, drop=True)
# aw_so = pd.DataFrame()
# for so_id in list(so[so['isbn'].isin(list(inv_aw['isbn']))].index.values):
#     for aw_inv_id in list(inv_aw[inv_aw['isbn'].isin(list(so['isbn']))].index.values):
#         if (inv_aw.iloc[aw_inv_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (inv_aw.iloc[aw_inv_id, :]['onhand'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0):
#             row_aw_so = pd.merge(inv_aw.iloc[[aw_inv_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_aw_so['allocate_qty'] = row_aw_so[['onhand','so_open_qty']].min( axis = 1, skipna = True)
#             inv_aw.loc[aw_inv_id,'onhand'] = inv_aw.loc[aw_inv_id,'onhand'] - row_aw_so.iloc[0,:]['allocate_qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_aw_so.iloc[0,:]['allocate_qty']
#             row_aw_so['orderstatus'] = 'READY TO SHIP'
#             row_aw_so['inv_aw_date_status'] = so.iloc[so_id, :]['so_date_status']
#             aw_so = aw_so.append(row_aw_so)

'\nRETAIL AW WAREHOUSE"\n\n'

In [17]:
# aw_so.head()

In [18]:
po.sort_values(['rowdeliverydate', 'po_whse', 'landedcost'], ascending=[True,False,True], inplace=True, ignore_index=True)
so.sort_values(['customer_drop_date', 'saleprice'], ascending=[True,False], inplace=True, ignore_index=True)

In [19]:
# real_sale = pd.DataFrame()
# real_sale = inv_so.append(po_so_good)

In [20]:
with pd.ExcelWriter('supply_chain/report.xlsx') as writer:
    inv_so.to_excel(writer,sheet_name = 'to_be_shipped', index= False)
    po[po['po_open_qty'] > 0 ].to_excel(writer, sheet_name = 'po_no_room', index= False)
    so[so['so_open_qty'] > 0 ].to_excel(writer, sheet_name = 'open_so', index= False)
    inventory_ww[inventory_ww['onhand'] > 0 ].to_excel(writer, sheet_name = 'extras_inventory', index= False)
    po_so.to_excel(writer,sheet_name = 'expected_sales', index= False)
    # aw_so.to_excel(writer,sheet_name = 'aw_reallocation', index= False) # Line when trying to make a sale from AW Warehouse.